# Creation variable test_enseigne a effectuer pour dedoublonner les lignes

Copy paste from Coda to fill the information

## Objective(s)

La siretisation repose sur une matrice de règles de gestion classée de manière ordonnée. Pour créer la matrice, il faut au préalable créer les variables nécéssaires à la création des tests. 

Le tableau ci dessous indique l'ensemble des tests a réaliser ainsi que leur dépendence.


Lors de cette US, nous allons créer 1 variable qui va permettre a la réalisation du test `test_enseigne`. La variable est la suivante:

- `list_enseigne_contain`: Variable boolean indiquant si le string de la variable `enseigne` est contenu dans la liste `list_enseigne`
* Il faut penser a garder la variable `row_id` 

## Metadata 

* Metadata parameters are available here: 
* US Title: Creation variable test_enseigne a effectuer pour dedoublonner les lignes
* Epic: Epic 5
* US: US 7
* Date Begin: 9/1/2020
* Duration Task: 0
* Status: active  
* Source URL: [US 07 Preparation tables et variables tests](https://coda.io/d/_dCtnoqIftTn/US-07-Preparation-tables-et-variables-tests_suFb9)
* Task type:
  * Jupyter Notebook
* Users: :
  * Thomas Pernet
* Watchers:
  * Thomas Pernet
* Estimated Log points:
  * One being a simple task, 15 a very difficult one
  *  1
* Task tag
  *  #sql-query,#regle-de-gestion,#preparation-test-enseigne
* Toggl Tag
  * #data-preparation
  
## Input Cloud Storage [AWS]

If link from the internet, save it to the cloud first

### Tables [AWS]

1. Batch 1:
  * Select Provider: Athena
  * Select table(s): ets_insee_inpi_siretisation
    * Select only tables created from the same notebook, else copy/paste selection to add new input tables
    * If table(s) does not exist, add them: Add New Table
    * Information:
      * Region: 
        * NameEurope (Paris)
        * Code: eu-west-3
      * Database: siretisation
      * Notebook construction file: 
        * [01_merge_ets_insee_inpi](https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/08_US_DATUM/01_merge_ets_insee_inpi.md)
    
## Destination Output/Delivery

  1. AWS
    1. Athena: 
      * Region: Europe (Paris)
      * Database: siretisation
      * Tables (Add name new table): ets_insee_inpi_list_enseigne
      * List new tables
      * ets_insee_inpi_list_enseigne

## Things to know (Steps, Attention points or new flow of information)

### Sources of information  (meeting notes, Documentation, Query, URL)

1. Jupyter Notebook (Github Link)
  1. md : https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/02_siretisation/07_pourcentage_siretisation_v3.md#creation-table-analyse


## Connexion serveur

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_athena import service_athena
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import os, shutil

path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/credential_AWS.json".format(parent_path)
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = 'eu-west-3')

region = 'eu-west-3'
bucket = 'calfdata'

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False) 

In [3]:
pandas_setting = True
if pandas_setting:
    cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Input/output

In [4]:
s3_output = 'inpi/sql_output'
database = 'inpi'

In [6]:
query = """
DROP TABLE siretisation.ets_insee_inpi_list_enseigne;
"""
s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

{'Results': {'State': 'SUCCEEDED',
  'SubmissionDateTime': datetime.datetime(2020, 9, 8, 10, 24, 51, 966000, tzinfo=tzlocal()),
  'CompletionDateTime': datetime.datetime(2020, 9, 8, 10, 24, 53, 122000, tzinfo=tzlocal())},
 'QueryID': 'e613e808-71ba-4675-b98d-393f0502e282'}

In [8]:
query = """
CREATE TABLE siretisation.ets_insee_inpi_list_enseigne
WITH (
  format='PARQUET'
) AS
SELECT 
row_id,
siret, 
enseigne,
list_enseigne,
contains(
  list_enseigne,
  enseigne
  ) AS list_enseigne_contain 
FROM siretisation.ets_insee_inpi
"""

s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

{'Results': {'State': 'SUCCEEDED',
  'SubmissionDateTime': datetime.datetime(2020, 9, 8, 10, 27, 58, 470000, tzinfo=tzlocal()),
  'CompletionDateTime': datetime.datetime(2020, 9, 8, 10, 28, 5, 370000, tzinfo=tzlocal())},
 'QueryID': '1529c2e9-9fad-499d-b1cb-6ae68bd1996d'}

In [19]:
query = """
SELECT * 
FROM siretisation.ets_insee_inpi_list_enseigne
WHERE list_enseigne_contain IS NOT NULL
LIMIT 10
"""
s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = 'exemple', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,row_id,siret,enseigne,list_enseigne,list_enseigne_contain
0,1905546,52119408400021,NaN,[KARMA],False
1,1905548,52119408400021,KARMA,[KARMA],True
2,1905559,52119493600022,WOODLAND - VIVREO,[WOODLAND - VIVREO],True
3,1905577,52119596600010,NaN,[AUTO SECURITE],False
4,1905578,52119596600010,AUTO SECURITE,[AUTO SECURITE],True
5,1905582,52119635200020,A LA CASCADE DE FLEURS,[A LA CASCADE DE FLEURS],True
6,1905586,52120143400011,SANDRINE COIFFURE,[SANDRINE COIFFURE],True
7,1905588,52120317400029,AR PUB,[AR PUB],True
8,1905591,52120360400025,BRAZIL BILLARD,[BRAZIL BILLARD],True
9,1905598,52120662300022,COGEST,[COGEST],True


# Test acceptance

1. Vérifier que le nombre de lignes est indentique avant et après la création des variables
2. Compter le nombre de lignes pour `list_enseigne_contain`
3. Compter le nombre d'occurences selon la cardinalité de `list_enseigne`

## 1. Vérifier que le nombre de lignes est indentique avant et après la création des variables

In [9]:
query = """
SELECT COUNT(*)
FROM siretisation.ets_insee_inpi_list_enseigne
"""
s3.run_query(
            query=query,
            database='siretisation',
            s3_output=s3_output,
  filename = 'count_ets_insee_inpi', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,_col0
0,11650040


In [10]:
query = """
SELECT COUNT(*)
FROM siretisation.ets_insee_inpi_list_enseigne
"""

s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = 'test_count_enseigne', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,_col0
0,11650040


## 2. Compter le nombre de lignes pour `list_enseigne_contain`


In [14]:
query = """
SELECT list_enseigne_contain, count(*) as cnt
       FROM siretisation.ets_insee_inpi_list_enseigne
       GROUP BY list_enseigne_contain

"""

s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = 'test_count_ligne_enseigne', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,list_enseigne_contain,cnt
0,False,667026
1,NaN,10151252
2,True,831762


## 3. Compter le nombre d'occurences selon la cardinalité de `list_enseigne`


In [15]:
query = """
SELECT CARDINALITY(list_enseigne) AS cardinality, COUNT(*) as cnt
FROM "siretisation"."ets_insee_inpi_list_enseigne"
GROUP BY CARDINALITY(list_enseigne)
ORDER BY cnt
"""

s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = 'cardinality', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

,cardinality,cnt
0,13.0,2
1,10.0,3
2,9.0,5
3,8.0,9
4,12.0,13
5,7.0,15
6,6.0,58
7,5.0,76
8,4.0,222
9,3.0,2396


# Generation report

In [16]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [20]:
def create_report(extension = "html", keep_code = False):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    if keep_code:
        os.system('jupyter nbconvert --to {} {}'.format(
    extension,notebookname))
    else:
        os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [21]:
create_report(extension = "html", keep_code = False)

Report Available at this adress:
 C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching\Notebooks_matching\Data_preprocessed\programme_matching\08_US_DATUM\Reports\04_test_enseigne.html
